<a href="https://colab.research.google.com/github/PeterStoreshaw/PeterStoreshaw.github.io/blob/master/Question_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Question 2, What Animal? A deep learning model that can identify 10 different animal species.

To Begin with we will use the code from fast AI
'bird or not'

In [ ]:
# It's a good idea to ensure you're running the latest version of any libraries you need.
# `!pip install -Uqq <libraries>` upgrades to the latest version of <libraries>
# NB: You can safely ignore any warnings or errors pip spits out about running as root or incompatibilities
import os
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

if iskaggle:
    !pip install -Uqq fastai

The following downloads duck duck go search and its upgraded version if needed.

In [ ]:
# Skip this cell if you already have duckduckgo_search installed
!pip install -Uqq duckduckgo_search

In [ ]:
!pip install --upgrade duckduckgo_search

Install Fastbook

In [ ]:
!pip install fastbook

Set up the image search

In [ ]:
from duckduckgo_search import ddg_images
from fastcore.all import *

def search_images(term, max_images=200): return L(ddg_images(term, max_results=max_images)).itemgot('image')

Let's start by searching for a bird photo 

In [ ]:
from fastbook import search_images_ddg 
urls = search_images_ddg('bird photos', max_images=1) 
urls[0]

...and then download a URL and take a look at it:

In [ ]:
from fastdownload import download_url
dest = 'bird.jpg'
download_url(urls[0], dest, show_progress=False)

from fastai.vision.all import *
im = Image.open(dest)
im.to_thumb(256,256)

Now let's do the same with "cat photos":
The cat photos should be pretty easy to identify, there are no other cat like animals and it is a specific species.

In [ ]:
download_url(search_images_ddg('cat photos', max_images=1)[0], 'cat.jpg', show_progress=False) 
Image.open('cat.jpg').to_thumb(256,256) 


Now let's do the same with "gorilla photos":
The Gorillas should be another easy one, though several of the gorilla photos include humans so it will be intersesting to see if using humans as the test photo brings up gorilla.

In [ ]:
download_url(search_images_ddg('gorilla photos', max_images=1)[0], 'gorilla.jpg', show_progress=False) 
Image.open('gorilla.jpg').to_thumb(256,256) 

Now let's do the same with "insect photos":
This on is interesting, insect is a very broad catagory. As such the 200 photos - the ones culled, may not be enough to train the system. 
This will be a good test. 

In [ ]:
download_url(search_images_ddg('insect photos', max_images=1)[0], 'insect.jpg', show_progress=False) 
Image.open('insect.jpg').to_thumb(256,256) 

Now let's do the same with "fish photos":
The fish is a similar catagory to insects, a varied group. However due to the water, it may be that the system has an easier time traning. 

In [ ]:
download_url(search_images_ddg('fish photos', max_images=1)[0], 'fish.jpg', show_progress=False) 
Image.open('fish.jpg').to_thumb(256,256) 

Now let's do the same with "kangaroo photos":
This is a very specific species and as such will help verify if the system can do the basics.

In [ ]:
download_url(search_images_ddg('kangaroo photos', max_images=1)[0], 'kangaroo.jpg', show_progress=False) 
Image.open('kangaroo.jpg').to_thumb(256,256) 

Now let's do the same with "koala photos":
Similar to kangaroo, Will help verify

In [ ]:
download_url(search_images_ddg('koala photos', max_images=1)[0], 'koala.jpg', show_progress=False) 
Image.open('koala.jpg').to_thumb(256,256) 

Now let's do the same with "elephant photos":
Same as previous should be easy to identify.

In [ ]:
download_url(search_images_ddg('elephant photos', max_images=1)[0], 'elephant.jpg', show_progress=False) 
Image.open('elephant.jpg').to_thumb(256,256) 

Now let's do the same with "lizards photos":
Now we get back to tricky, this is a broad group, that has similarities to the frogs. So it along with frogs, fish and insects will determine how rigorous the program is.

In [ ]:
download_url(search_images_ddg('lizard photos', max_images=1)[0], 'lizard.jpg', show_progress=False) 
Image.open('lizard.jpg').to_thumb(256,256) 

Now let's do the same with "frog photos":
Similar to lizards, though a more specific group. Issue may be that there are lots of cartoons in the traning folder.

In [ ]:
download_url(search_images_ddg('frog photos', max_images=1)[0], 'frog.jpg', show_progress=False) 
Image.open('frog.jpg').to_thumb(256,256) 

Our searches seem to be giving reasonable results, so let's grab 200 examples of each and save each group of photos to a different folder:

If using google colab we can just grab those photos already downloaded.
If runing for the first time skip this step 

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

!unzip '/content/drive/MyDrive/Q2t.zip'

path = Path('Q2') 

If the previous step was run and successful SKIP THIS STEP.

In [ ]:
searches = 'cat', 'gorilla', 'insect', 'fish', 'koala', 'kangaroo', 'lizard', 'frog', 'elephant', 'bird' 
path = Path('Q2') 
from time import sleep 

for o in searches: 
    dest = (path/o) 
    dest.mkdir(exist_ok=True, parents=True) 
    download_images(dest, urls=search_images_ddg(f'{o} photo')) 
    sleep(10)  # Pause between searches to avoid over-loading server 
    download_images(dest, urls=search_images_ddg(f'{o} sun photo')) 
    sleep(10) 
    download_images(dest, urls=search_images_ddg(f'{o} shade photo')) 
    sleep(10) 
    resize_images(path/o, max_size=400, dest=path/o) 


## Step 2: Train our model

Some photos might not download correctly which could cause our model training to fail, so we'll remove them:

After this step I went into the folders and manualy deleted all of the phots that do not fit. There were so so many lamps. The process is detailed in my blog.

In [ ]:
failed = verify_images(get_image_files(path))
failed.map(Path.unlink)
len(failed)

Create the data block.

In [ ]:
dls = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=[Resize(192, method='squish')]
).dataloaders(path)

dls.show_batch(max_n=6)

If running in colab, once all photos downloaded. Run this box, otherwise skip.

In [ ]:
from google.colab import drive

# mount will create a folder at the left folder section on the page, but it will take a minute or so to see it

drive.mount('/content/drive') 

!zip -r Q2.zip Q2

!cp Q2.zip /content/drive/My\ Drive/ 

Run the learner

In [ ]:
learn = vision_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(3)

## Step 3: Check Accuracy

Create and plot a confusion matrix


The confusion matrix will help identify the accuracy. 
It works by finding the true negative, true positive, false negative and false positive for each selection. 
So, if we predicted cat, and it was a cat that is a true positive. 
The matrix this code will create just looks at true positives and incorrect choices. If the majority of diagonal line from left to right is all 100 percent we know that the system is perfectly accurate. 


In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

Show the worst 8 photos for confusion.
 This shows us the 8 most troublesome photos that the sytem is stugling to correctly identify.

In [ ]:
interp.plot_top_losses(8, nrows=1)


A t-SNE function taken from playing-with-fastai /00 - Rossman Embeddings.ipynb

In [ ]:
def idx2EmbPlot(idx,perplexity=30,early_exaggeration=12.0, learning_rate=200.0,ax=None):
    Embedding_matrix = to_np(next(m.get_layer_groups()[0][idx].parameters()))
    from sklearn.manifold import TSNE
    X_embedded = TSNE(n_components=2, perplexity=perplexity, early_exaggeration=early_exaggeration, 
                      learning_rate=learning_rate,n_iter=1000, n_iter_without_progress=300, 
                      min_grad_norm=1e-07,metric='euclidean', init='random', verbose=0, 
                      random_state=42, method='barnes_hut', angle=0.5).fit_transform(Embedding_matrix)
    
    annotation = np.append(joined_samp[joined_samp.columns[idx]].cat.categories.values,"Other")
    
    
    ax.scatter(X_embedded[:,0],X_embedded[:,1],c=range(len(annotation)),alpha=0.4)
    for xy,target in zip(X_embedded,annotation):
        if isinstance(target,np.str_) or isinstance(target,str):
            text = target
        else:
            text = "%d" % target
        ax.annotate(s=text,xy=(xy[0],xy[1]))
        ax.set_title(joined_samp.columns[idx])

## Step 4: Prediction

In [ ]:
guess,_,probs = learn.predict(PILImage.create('images.jpg'))
print(f"This is a: {guess}.")
